# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [30]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#

import sqlite3
import pandas as pd
import numpy as np

#Read data
sq_df = pd.read_csv("https://bit.ly/3BPcobU", encoding = "latin1")
sq_df.head()


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012.0,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012.0,20000.0
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012.0,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012.0,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011.0,20000.0


In [31]:
#missing values counts 
sq_df = pd.read_csv("https://bit.ly/3BPcobU", encoding = "latin1", chunksize = 5000)

for chunk in sq_df:
   
    missing_value_counts = chunk.isnull().sum()
    
    print(missing_value_counts)

company_permalink            0
company_name                 0
company_category_code       52
company_country_code         0
company_state_code          53
company_region               0
company_city                64
investor_permalink           0
investor_name                0
investor_category_code    2557
investor_country_code      778
investor_state_code       1371
investor_region              0
investor_city              900
funding_round_type           0
funded_at                    0
funded_month                 0
funded_quarter               0
funded_year                  0
raised_amount_usd          653
dtype: int64
company_permalink            0
company_name                 0
company_category_code       51
company_country_code         0
company_state_code          43
company_region               0
company_city                45
investor_permalink           0
investor_name                0
investor_category_code    5000
investor_country_code      261
investor_state_code       

In [32]:
#chunks of 5000
sq_df = pd.read_csv("https://bit.ly/3BPcobU", encoding = "latin1", chunksize = 5000)
chunk_sq = 0
memory_usage = 0
initial_memory_usage = 0

for chunk in sq_df:
  memory_usage = chunk.memory_usage(deep=True).sum()/(1024*1024)
  initial_memory_usage +=memory_usage
  chunk_sq += 1
  print(f'Chunk {chunk_sq} Memory Usage: {memory_usage}MB')

print(f'\ntotal memory  of all chunks combined is:{initial_memory_usage}MB')

Chunk 1 Memory Usage: 5.579240798950195MB
Chunk 2 Memory Usage: 5.528232574462891MB
Chunk 3 Memory Usage: 5.535050392150879MB
Chunk 4 Memory Usage: 5.528193473815918MB
Chunk 5 Memory Usage: 5.5243377685546875MB
Chunk 6 Memory Usage: 5.553427696228027MB
Chunk 7 Memory Usage: 5.531436920166016MB
Chunk 8 Memory Usage: 5.5096588134765625MB
Chunk 9 Memory Usage: 5.396121025085449MB
Chunk 10 Memory Usage: 4.639497756958008MB
Chunk 11 Memory Usage: 2.6637144088745117MB

total memory  of all chunks combined is:56.988911628723145MB


**columns to drop**
1.  *funded_month* *funded_year*  because the are redudant 
2.  investor_category_code contains highest missing values 






## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [33]:
# Identify the numeric columns we can represent using more space efficient types.

sq1_df = pd.read_csv("https://bit.ly/3BPcobU", encoding = "latin1", chunksize=5000)

columns_to_drop = ['investor_category_code', 'funded_month', 'funded_year']

total_memory = 0


for chunk in sq1_df:
   
    chunk = chunk.drop(columns_to_drop, axis=1)

  
    chunk[chunk.select_dtypes(include=['float']).columns] = chunk.select_dtypes(include=['float']).apply(pd.to_numeric, downcast='float')
    chunk[chunk.select_dtypes(include=['int']).columns] = chunk.select_dtypes(include=['int']).apply(pd.to_numeric, downcast='integer')
    

    chunk_memory = chunk.memory_usage(deep=True).sum() /  (1024 ** 2)
    print("Chunk memory footprint:", chunk_memory, "MB")

  
    total_memory += chunk_memory

# total memory usage
print("Total memory usage:", total_memory, "MB")

Chunk memory footprint: 4.988473892211914 MB
Chunk memory footprint: 5.127689361572266 MB
Chunk memory footprint: 5.134507179260254 MB
Chunk memory footprint: 5.127650260925293 MB
Chunk memory footprint: 5.1237945556640625 MB
Chunk memory footprint: 5.152884483337402 MB
Chunk memory footprint: 5.130985260009766 MB
Chunk memory footprint: 5.1091156005859375 MB
Chunk memory footprint: 4.995577812194824 MB
Chunk memory footprint: 4.181734085083008 MB
Chunk memory footprint: 2.4009580612182617 MB
Total memory usage: 52.47337055206299 MB


Total memory reduced by 4 MB

In [34]:
rem_columns = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'funding_round_type',
        'company_category_code', 'funded_at', 'company_state_code',
        'company_region', 'company_permalink', 'company_name', 
        'company_country_code', 'company_city', 'investor_state_code', 
        'funded_quarter', 'raised_amount_usd']

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [35]:
# Your code goes here

import sqlite3
import pandas as pd
conn = sqlite3.connect('mesobase.db')

sq1_pd = pd.read_csv('https://bit.ly/3BPcobU', chunksize=5000,encoding='latin1',)

for chunk in sq1_pd:
 
 chunk['raised_amount_usd']=chunk['raised_amount_usd'].astype('float')
 chunk['month'] = chunk['funded_month'].str.split('-').str[-1]
 chunk['month']= pd.to_numeric(chunk['month'],downcast = 'signed')
 chunk.drop(['funded_month'], axis = 1,inplace=True)
 chunk.to_sql("mesobase", conn, if_exists='append', index=False)

sq2_df = pd.read_sql('PRAGMA table_info(mesobase);', conn)
print(sq2_df)


confirm_df =pd.read_sql('''SELECT * FROM mesobase;''', conn)
confirm_df

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_quarter,funded_year,raised_amount_usd,month
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,None,USA,NY,New York,New York,series-a,2012-10-30,2012-Q4,2012.0,2000000.0,10.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-Q1,2012.0,20000.0,1.0
2,/company/utap,uTaP,messaging,USA,None,United States - Other,None,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-Q1,2012.0,20000.0,1.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-Q1,2012.0,20000.0,2.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-Q3,2011.0,20000.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158605,/company/garantia-data,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,/person/zohar-gilon,Zohar Gilon,None,None,None,unknown,None,series-a,2012-08-08,2012-Q3,2012.0,3800000.0,8.0
158606,/company/duda-mobile,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,/person/zohar-gilon,Zohar Gilon,None,None,None,unknown,None,series-c+,2013-04-08,2013-Q2,2013.0,10300000.0,4.0
158607,/company/sitebrains,SiteBrains,software,USA,CA,SF Bay,San Francisco,/person/zohar-israel,zohar israel,None,None,None,unknown,None,angel,2010-08-01,2010-Q3,2010.0,350000.0,8.0
158608,/company/comprehend-systems,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,/person/zorba-lieberman,Zorba Lieberman,None,None,None,unknown,None,series-a,2013-07-11,2013-Q3,2013.0,8400000.0,7.0


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [36]:

# Your code goes here
grouped_df = confirm_df.groupby("company_name").sum()
sort_df = grouped_df.sort_values("raised_amount_usd", ascending=False)
total_funds = sort_df["raised_amount_usd"].sum()
top_10_funds = sort_df.iloc[:int(len(sort_df) * 0.1)]["raised_amount_usd"].sum()
top_10_proportion = 100* top_10_funds / total_funds
top_1_funds = sort_df.iloc[:int(len(sort_df) * 0.01)]["raised_amount_usd"].sum()
top_1_proportion =100* top_1_funds / total_funds


bottom_10_funds = sort_df.iloc[-int(len(sort_df) * 0.1):]["raised_amount_usd"].sum()
bottom_10_proportion = 100*bottom_10_funds / total_funds
bottom_1_funds = sort_df.iloc[-int(len(sort_df) * 0.01):]["raised_amount_usd"].sum()
bottom_1_proportion = 100*bottom_1_funds / total_funds

print("top 10% raised :", top_10_proportion,"%")
print("top 1% rasied :", top_1_proportion,"%")
print(" bottom 10% raised :", bottom_10_proportion,'%')
print("bottom  1%: raised ", bottom_1_proportion,'%')



top 10% raised : 67.12772709906496 %
top 1% rasied : 26.217737281766546 %
 bottom 10% raised : 0.0 %
bottom  1%: raised  0.0 %




1.   top 10% raised : 67.12772709906496 %
2.  top 1% rasied : 26.217737281766546 %
3. bottom 10% and 1% contribution was negligable 





In [37]:
confirm_df.groupby(by='company_category_code').raised_amount_usd.sum().sort_values(ascending=False)

company_category_code
biotech             3.311893e+11
software            2.192536e+11
mobile              1.943321e+11
cleantech           1.581157e+11
enterprise          1.375828e+11
web                 1.204298e+11
medical             7.610132e+10
advertising         7.522999e+10
ecommerce           6.770166e+10
network_hosting     6.725905e+10
semiconductor       6.707501e+10
hardware            6.283635e+10
games_video         5.822895e+10
analytics           4.123270e+10
security            3.944961e+10
social              3.459129e+10
finance             3.360061e+10
health              2.636790e+10
nanotech            2.537496e+10
public_relations    2.344570e+10
automotive          2.171775e+10
other               1.879932e+10
search              1.754337e+10
education           1.650693e+10
manufacturing       1.548832e+10
news                1.475029e+10
fashion             1.170661e+10
travel              9.127179e+09
messaging           8.473994e+09
consulting          5

biotech attracted most investments

In [38]:
confirm_df.groupby(by='investor_name').raised_amount_usd.sum().sort_values(ascending=False)

investor_name
Kleiner Perkins Caufield & Byers              3.365348e+10
New Enterprise Associates                     2.907763e+10
Accel Partners                                1.941638e+10
Goldman Sachs                                 1.912638e+10
Sequoia Capital                               1.811821e+10
                                                  ...     
Charles Crawford                              0.000000e+00
New York City Economic Development Council    0.000000e+00
Michael Milken                                0.000000e+00
Robin Cremeens                                0.000000e+00
Onvelop                                       0.000000e+00
Name: raised_amount_usd, Length: 10465, dtype: float64

The investor who contributed the most money is: Kleiner Perkins Caufield & Byers

In [39]:

grouped_df = confirm_df.groupby("funding_round_type").count()
sort_df = grouped_df.sort_values("company_name", ascending=False)
most_popular= sort_df.index[0]
least_popular = sort_df.index[-1]

# Print the result
print(" most popular funding round :", most_popular)
print(" least popular funding round :", least_popular)

 most popular funding round : series-a
 least popular funding round : crowdfunding
